In [1]:
import scipy as sp
import numpy as np

In [8]:
from scipy.stats import binom

# Bravo
### p1=0.7

In [33]:
# Explore bravo decision boundary
def bravo_boundary(c, m, p0, p1):
    return (c - m * np.log(2 * (1 - p1))) / np.log(p1/(1-p1))

def bravo_boundary_inv(ym, m, p0, p1):
    return np.log(p1/(1-p1)) * ym + m * np.log(2*(1-p1))

def alpha_from_c(c): return 1/np.exp(c)

In [35]:
alpha_from_c(bravo_boundary_inv(1037, 2000, 0.5, 0.7))

1.2751840802046822e+62

In [32]:
bravo_boundary(-143, 2000, 0.5, 0.7)

1037.00397299534

In [9]:
binom.ppf(0.95, 2000, 0.5)

1037.0

In [23]:
c = -143
alpha = 1/np.exp(c)
alpha

1.2708986318302189e+62

### p1=0.55

In [9]:
binom.ppf(0.95, 2000, 0.5)


1037.0

In [36]:
alpha_from_c(bravo_boundary_inv(1037, 2000, 0.5, 0.55))

13.811756108118056